In [ ]:
import torch
from torch import nn
import torchviz
import dnaencoder
import vit
import scipy.io
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dataset
import dnaencoder
from importlib import reload
import mat

/home/amintehrani/.pyenv/versions/3.12.8/envs/torchgeo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
MAT_FILE_PATH = './insect_dataset.mat'

mat = scipy.io.loadmat(MAT_FILE_PATH)
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'all_images', 'all_dnas', 'all_labels', 'all_dnas_norepeat', 'all_dna_labels_norepeat', 'all_boldids', 'train_loc', 'val_seen_loc', 'val_unseen_loc', 'test_seen_loc', 'test_unseen_loc', 'species2genus', 'described_species_labels_train', 'described_species_labels_trainval', 'all_dna_features_cnn_original', 'all_image_features_resnet', 'all_image_features_gan', 'all_dna_features_cnn_new', 'all_string_dnas'])

In [4]:
species2genus = mat['species2genus']-1

# group species by genus

genus_species = dict()
max_specie_in_genus = 0
for genus_id, genus in pd.DataFrame(species2genus, columns=['genus']).groupby('genus'):
    specie_indices = genus.index.tolist()
    genus_species[genus_id] = specie_indices
    if len(specie_indices) > max_specie_in_genus:
        max_specie_in_genus = len(specie_indices)

print(len(genus_species))
print("Max specie in genus: ", max_specie_in_genus)


372
Max specie in genus:  23


In [ ]:
# genus = lambda s: mat['species2genus'][s]-1
# genus(156)

array([62])

In [36]:
# group labels count
pd.Series(mat['all_labels'].squeeze()).value_counts()


1038    759
977     540
1039    361
418     292
979     292
       ... 
529       4
578       2
530       2
533       2
156       1
Name: count, Length: 1050, dtype: int64

In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=512)
all_dna_features_cnn_pca = pca.fit_transform(mat['all_dna_features_cnn_new'])


In [6]:
pca = PCA(n_components=512)
all_image_features_gan_pca = pca.fit_transform(mat['all_image_features_gan'])

In [84]:
mat['species2genus'].shape

(1050, 1)

In [ ]:
x = np.array(list(map(lambda s: len(s.strip()), mat['all_string_dnas'])))
np.unique(x).size

120

In [43]:

all_dna_len = list(map(lambda s: len(s.strip()), mat['all_string_dnas']))
dna_str_len_mapping: dict[int,int] = {}

def dna_str_len_to_int(s_len):
    if s_len not in dna_str_len_mapping:
        dna_str_len_mapping[s_len] = len(dna_str_len_mapping)
    return dna_str_len_mapping[s_len]

def all_dna_len_token():
    return list(map(dna_str_len_to_int, all_dna_len))

all_dna_len_tokens = list(map(dna_str_len_to_int, all_dna_len))
print(list(zip(all_dna_len, all_dna_len_tokens)))

[(658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (666, 1), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (638, 2), (638, 2), (638, 2), (658, 0), (700, 3), (679, 4), (659, 5), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0), (658, 0),

In [ ]:
deviceGPU = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
deviceCPU = torch.device("cpu")

In [87]:
# import models
# reload(models)
# from models import AttentionFusion, SpeciePredictor, Decoder, GenusPredictor



# fusion = AttentionFusion(512, 512, 512, num_heads=8).to(device)
# genus_predictor = GenusPredictor(mat['species2genus'], fusion).to(device)
# fusion.to(deviceGPU)
genus_predictor.to(deviceGPU)
genus_predictor.fit(all_dna_len_tokens,
            all_dna_features_cnn_pca, 
              all_image_features_gan_pca, 
              mat['all_labels'].squeeze(), 
              mat['val_seen_loc'].squeeze(), 
              mat['train_loc'].squeeze(), 
              1000, 
              lr=0.005,
              eval_frequency=2,
              device=deviceGPU)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
genus_predictor.load_state_dict(torch.load('output/Tue Aug 12 20:36:21 2025_best_genus_predictor.pt'))
genus_predictor.to(deviceCPU)

GenusPredictor(
  (fusion_encoder): AttentionFusion(
    (dna_len_emb): Embedding(120, 16)
    (proj_dna): Linear(in_features=512, out_features=496, bias=True)
    (proj_img): Linear(in_features=512, out_features=512, bias=True)
    (ffn): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
    )
  )
  (decoder): Decoder(
    (ffn): Sequential(
      (0): Linear(in_features=512, out_features=744, bias=True)
      (1): Sigmoid()
      (2): Linear(in_features=744, out_features=372, bias=True)
    )
  )
  (criterion): CrossEntropyLoss()
)

In [ ]:
import models
reload(models)
from models import AttentionFusion, SpeciePredictor, Decoder, GenusPredictor

specie_predictor = SpeciePredictor(mat['species2genus'],genus_species, genus_predictor).to(deviceCPU)
specie_predictor.fit(
    all_dna_len_tokens,
    all_dna_features_cnn_pca, 
    all_image_features_gan_pca, 
    mat['all_labels'].squeeze(), 
    mat['val_seen_loc'].squeeze(), 
    mat['train_loc'].squeeze(), 
    200, 
    lr=0.005,
    eval_frequency=10,
    freeze_genus=True,
    teacher_force=True,
    device=deviceCPU)

tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([135., 135., 135.,  ..., 135., 135., 135.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ..., 311., 311., 311.])
tensor([234., 234., 234.,  ...,

{'train_loss': [3.2404184341430664,
  3.128836154937744,
  3.017695903778076,
  2.906017541885376,
  2.793914794921875,
  2.6818089485168457,
  2.570011615753174,
  2.4594290256500244,
  2.3511242866516113,
  2.244725465774536,
  2.1398942470550537,
  2.0379819869995117,
  1.9407556056976318,
  1.8498947620391846,
  1.7642860412597656,
  1.6841204166412354,
  1.6098366975784302,
  1.5398410558700562,
  1.4734737873077393,
  1.4095193147659302,
  1.3482303619384766,
  1.2886369228363037,
  1.2305690050125122,
  1.1740379333496094,
  1.1182886362075806,
  1.0635336637496948,
  1.0094034671783447,
  0.9559038281440735,
  0.9032287001609802,
  0.8513595461845398,
  0.8004623651504517,
  0.7509310841560364,
  0.7031145095825195,
  0.6573391556739807,
  0.6136695742607117,
  0.5722282528877258,
  0.5330378413200378,
  0.49608689546585083,
  0.461538165807724,
  0.42951318621635437,
  0.3999958038330078,
  0.37290820479393005,
  0.3481047749519348,
  0.3254525363445282,
  0.30481278896331787,

In [26]:
all_dna_features_cnn_pca.shape

(32424, 768)

In [ ]:
np.unique(mat['all_labels'])

array([   1,    2,    3, ..., 1048, 1049, 1050], shape=(1050,))

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DNAImageDecoder(nn.Module):
    def __init__(self, N_dna, N_image, d_model=128, num_heads=4, num_classes=10):
        super().__init__()
        
        # Project DNA and image embeddings into same space
        self.dna_proj = nn.Linear(N_dna, d_model)
        self.img_proj = nn.Linear(N_image, d_model)
        
        # Self-attention mechanism
        self.attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)
        
        # Feed-forward layer after attention
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, num_classes)
        )
        
    def forward(self, dna_emb, img_emb):
        """
        dna_emb: (batch_size, N_dna)
        img_emb: (batch_size, N_image)
        """
        # Project to same dimension
        dna_token = self.dna_proj(dna_emb).unsqueeze(1)  # (batch, 1, d_model)
        img_token = self.img_proj(img_emb).unsqueeze(1)  # (batch, 1, d_model)
        
        # Sequence: [DNA, Image]
        seq = torch.cat([dna_token, img_token], dim=1)  # (batch, 2, d_model)
        
        # Self-attention
        attn_out, _ = self.attn(seq, seq, seq)  # (batch, 2, d_model)
        
        # Pooling — use first token (DNA) or mean-pool
        pooled = attn_out.mean(dim=1)  # (batch, d_model)
        
        # Classification
        logits = self.ffn(pooled)  # (batch, num_classes)
        return logits


In [43]:
ma = nn.MultiheadAttention(768, 4)
attn_output, attn_output_weights = (ma(torch.rand(1, 1, 768), torch.rand(1, 1, 768), torch.rand(1, 1, 768)))


In [47]:
attn_output_weights

tensor([[[1.]]], grad_fn=<MeanBackward1>)

In [ ]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
dna_emb = dnaencoder.model(**dnaencoder.tokenizer(dna, return_tensors='pt',))


odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch

# Load pretrained ViT model
model_name = "google/vit-base-patch16-224"
imageprocessor = AutoImageProcessor.from_pretrained(model_name)
vitmodel = AutoModel.from_pretrained(model_name)
vitmodel.eval()

# Put model on GPU if available
deviceCPU = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vitmodel.to(deviceCPU)

def get_vit_embedding(image_np, model_name="google/vit-base-patch16-224", device = torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    inputs = imageprocessor(images=image_np, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = vitmodel(**inputs)
    
    return outputs.last_hidden_state[:, 0]  # CLS token


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
dna_emb.pooler_output.shape

torch.Size([1, 768])